In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_array = cv2.imread('data/train/3/Training_185749.jpg')

In [ ]:
img_array.shape

In [ ]:
plt.imshow(img_array)

In [ ]:
classes = ['0', '1', '2','3', '4', '5', '6']
# name of the classes

In [ ]:
train_dir = 'data/train'
test_dir = 'data/test'

In [ ]:
img_size = 224
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
new_array.shape

In [ ]:
# reading all the images and converting them in to array of new size

train_data = []

def create_training_data():
    for i in classes:
        path = os.path.join(train_dir, i)
        class_num = classes.index(i) # labels
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                train_data.append([new_array, class_num])
            except Exception as e:
                pass

In [ ]:
create_training_data()

In [ ]:
print(len(train_data))

In [ ]:
# shuffling the data
import random
random.shuffle(train_data)

In [ ]:
X = [] # features
Y = [] # labels

for feature, label in train_data:
    X.append(feature)
    Y.append(label)

In [ ]:
# changing the shape for deep learning model
X = np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
X.shape

In [ ]:
Y = np.array(Y)

In [ ]:
Y.shape

## Model building

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

In [ ]:
# transfer learning (tuning weights)

base_input = model.layers[0].input
base_output = model.layers[-2].output

In [ ]:
base_output

In [ ]:
final_output = layers.Dense(128)(base_output) # adding new layer after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) # activation function
final_output = layers.Dense(64)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)

In [ ]:
final_output

In [ ]:
new_model = keras.Model(inputs=base_input, outputs=final_output)

In [ ]:
new_model.summary()

In [ ]:
# compiling the model

new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
# training the model

new_model.fit(X, Y, epochs=30)